# BERT
This part has been implemented by Amir Sadeghi (github: amirsdg77)

# Preparing

In [ ]:
!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1akqFYHLyT25JD9PuL36LaX3oGiA_dAWG',
                                    dest_path='/content/data.csv',
                                    unzip=True)

Unzipping...

/usr/local/lib/python3.7/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1akqFYHLyT25JD9PuL36LaX3oGiA_dAWG" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
!pip install --quiet transformers
!pip install --quiet torch
!pip install --quiet python-bidi
!pip -q install --quiet hazm
!pip -q install --quiet clean-text[gpl]
!pip install --quiet bs4
!pip install --quiet sklearn
!pip install --quiet collections
!pip install --quiet textwrap
!pip install --quiet pylab
!pip install --quiet matplotlib
!pip install --quiet seaborn
!pip install --quiet sentencepiece

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 33.3 MB/s 
     |████████████████████████████████| 3.3 MB 41.1 MB/s 
     |████████████████████████████████| 596 kB 47.3 MB/s 
     |████████████████████████████████| 61 kB 429 kB/s 
     |████████████████████████████████| 316 kB 5.3 MB/s 
     |████████████████████████████████| 233 kB 1.1 MB/s 
     |████████████████████████████████| 1.4 MB 44.4 MB/s 
     |████████████████████████████████| 170 kB 5.2 MB/s 
     |████████████████████████████████| 64 kB 2.8 MB/s 
     |████████████████████████████████| 235 kB 48.3 MB/s 
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
ERROR: Could not find a version that satisfies the requirement textwrap (from versions: none)
ERROR: No matching distribution found for textwrap
ERROR: Could not find a version that satisfies the requirement pylab (from

In [ ]:
import transformers
from transformers import AlbertModel, AdamW, get_linear_schedule_with_warmup, AutoTokenizer, AutoModel, AlbertTokenizer

import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [ ]:
import json
df = pd.read_csv("data.csv")
df.head()

,cleaned,Labels
0,one person hour retweet amp follow plexbtc amp...,2
1,hey join network call surveyj earn today get p...,0
2,btc big bid alert avg bid volum avg price powe...,0
3,btc trade plan week binanc btcusdt tradehubng ...,0
4,coldmandrak found bitcoin user vault locat joi...,2


In [ ]:
df.Labels.unique()

array([2, 0, 1])

In [ ]:
df[df['Labels'] == 0]

,cleaned,Labels
1,hey join network call surveyj earn today get p...,0
2,btc big bid alert avg bid volum avg price powe...,0
3,btc trade plan week binanc btcusdt tradehubng ...,0
5,twittersupport effort attack govern el bitcoin...,0
6,found bitcoin user vault locat join play coinh...,0
...,...,...
9988,hey join network call surveyj earn today get p...,0
9991,xrp hit listen fud patienc make gain day bette...,0
9992,hey spook jasonploweri get hous order play phi...,0
9993,annnd wrap look like got juic left tho k today...,0


# data preprocessing

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import string

from cleantext import clean
import codecs,re

def cleaning(text):
  text = str(text)
  text = re.sub(r'#', ' ', text)
  text = re.sub(r'_', ' ', text)
  text = text.replace('\u200c', ' ')
  text = re.sub(r'[^\w\s]',' ',text)
  text = re.sub(r'\@\w*', ' ', text)
  text = re.sub(r'\d+', ' ', text)
  text = re.sub(r'$', ' ', text)
  text = re.sub(r'<(.*?)>', ' ', text)
  text = re.sub(r'https?:\/\/.*[\r\n]*', ' ', text)
  text=re.sub(r'[\·\♦\٭\\,\^\|\˝\٬\’\”\‹\▪\○¼ç½éêüəıœ™Ááàäāćíłñū©ٰٔ]',' ',text)
  text=re.sub(r'[ًٌٍَُِّْ]',' ',text)
  text=re.sub(r'[]',r' ',text)
  text=re.sub(r'[\–\—…°≈≠±≤≥\−×÷√٪→←↔↑↓\#\٫]',u' ',text) 
  text=text.replace(u'•',u' ').replace(u'ˈ',u' ').replace(u'؛',u' ').replace(u'/',u' ').replace(u'ۀ',u'هٔ').replace(u"﴿",u' ').replace(u"﴾",u' ').replace(u"'",u' ').replace(u'\\',u' ').replace(u'[',u' ').replace(u']',u' ').replace(u'?',u' ').replace(u'؟',u' ').replace(u')',u' ').replace(u'_',u' ').replace(u'(u',u' ').replace(u'}',u' ').replace(u'{',u' ').replace(u'.',u' ').replace(u'>',u' ').replace(u'<',u' ')
  text=text.replace(u'`',u' ').replace(u'\t',u' ').replace(u'=',u' ').replace(u'»',u' ').replace(u'«',u' ').replace(u'~',u' ').replace(u'!',u' ').replace(u'@',u' ').replace(u'$',u' ').replace(u',u',u' ').replace(u'%',u' ').replace(u'،',u' ').replace(u'-',u' ').replace(u';',u' ').replace(u':',u' ').replace(u'*',u' ').replace(u'"',u' ').replace(u'&',u' ').replace(u'#',u' ').replace(u'+',u' ')
  text=re.sub(r'[\n\r]{2,}',u'\n',text)
  text = re.sub(u'(\u202A|\u202B|\u202C|\u202D|\u202E|\u200F|\uFEFF|\u2003|\¬|\­)',u'\u200C', text)#حذف کارکترهای تغییرجهت
  text = re.sub(u'‌{2,}', u'‌', text) # پشت‌سرهم
  text=text.replace(u'­',u' ').replace(u'­',u' ').replace(u'ـ',u' ').replace(u'ـ',u' ').replace(u'ـ',u' ').replace(u'ـ',u' ').replace(u'',u' ')
  text=text.replace(u'',u' ')
  text = re.sub(u'(\u00A0)',u' ', text).replace(u'(',u' ').replace(u')',u' ')
  text=text.replace(u'    ',u' ').replace(u'    ',u' ').replace(u'   ',u' ').replace(u'  ',u' ').replace(u'  ',u' ').replace(u'  ',u' ')
  return text.strip()

# def remove_stopwords(text):
#   tokens = word_tokenize(text)
#   persian_stopwords = StopWords

#   results = [w for w in tokens if w not in persian_stopwords and w not in string.punctuation]
  
#   return ' '.join(results)

# def stemming(text):
#   tokens = word_tokenize(text)
#   persian_stemmer = Stemmer()

#   results = [persian_stemmer.stem(w) for w in tokens]

#   return ' '.join(results)

# def lemmatizing(text):
#   wordlist = []
#   lemmatizer = WordNetLemmatizer()
  
#   sentences = sent_tokenize(text)

#   for sentence in sentences:
#         words=word_tokenize(sentence)
#         for word in words:
#             wordlist.append(lemmatizer.lemmatize(word))

#   return ' '.join(wordlist)

# def normalize(text):
#   # result = lemmatizing(text)
#   result = stemming(text)
#   result = remove_stopwords(result)

#   return result


In [ ]:
def plus(num):
  return num + 1

In [ ]:
df['cleaned'] = df.text.apply(cleaning)

In [ ]:
df['Labels'] = df.affect.apply(plus)

In [ ]:
df.head()

,text,rate,affect,cleaned,Labels
0,Why ‘Cardano is just teasing us’\n\n💡 https://...,0.002199,0,Why Cardano is just teasing us \n https t co d...,1
1,#Thailand #Boutiquehotels For Sale! \n#willroa...,0.002199,0,Thailand Boutiquehotels For Sale \n willroadho...,1
2,LUNALAND ($LLN) will burn coin supply until we...,0.002199,0,LUNALAND LLN will burn coin supply until we re...,1
3,#Ilovebitcoin because people never assume that...,0.002199,0,Ilovebitcoin because people never assume that ...,1
4,$bsc $btc #BNB #Bitcoin #BSC #EarnBNB #BTC htt...,0.002199,0,bsc btc BNB Bitcoin BSC EarnBNB BTC https t co...,1


# Model (Fine-Tune)

## perparing

In [ ]:
class_names = ['negative', 'neutral', 'positive']

In [ ]:
model_name = 'bert-base-uncased'
# model_name = "HooshvareLab/albert-fa-zwnj-base-v2"

In [ ]:
from transformers import BertModel, BertTokenizer

bert_model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
# token_lens = []

# for txt in df.cleaned:
#   tokens = tokenizer.encode(txt, max_length=512)
#   token_lens.append(len(tokens))

In [ ]:
# sns.distplot(token_lens)
# plt.xlim([0, 256]);
# plt.xlabel('Token count');
# max(token_lens)

In [ ]:
RANDOM_SEED = 42
MAX_LEN = 120 # should be changed after determining the max length

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

In [ ]:
class CommentDataset(Dataset):

  def __init__(self, comments, targets, tokenizer, max_len):
    self.comments = comments
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.comments)
  
  def __getitem__(self, item):
    comment = str(self.comments[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      comment,
      truncation=True,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'comment_text': comment,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = CommentDataset(
    comments=df.cleaned.to_numpy(),
    targets=df.Labels.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=2
  )

In [ ]:
BATCH_SIZE = 32

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

data = next(iter(train_data_loader))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

In [ ]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained(model_name, return_dict=False)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
model = SentimentClassifier(len(class_names))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)

In [ ]:
F.softmax(model(input_ids, attention_mask), dim=1)

tensor([[0.2352, 0.3115, 0.4532],
        [0.2474, 0.3317, 0.4209],
        [0.1745, 0.4186, 0.4069],
        [0.1545, 0.3083, 0.5372],
        [0.2337, 0.3939, 0.3724],
        [0.3277, 0.3321, 0.3402],
        [0.2286, 0.3659, 0.4055],
        [0.2360, 0.4458, 0.3182],
        [0.2676, 0.3449, 0.3875],
        [0.2004, 0.3351, 0.4645],
        [0.1517, 0.4712, 0.3771],
        [0.2729, 0.4168, 0.3103],
        [0.1621, 0.4055, 0.4324],
        [0.2497, 0.4348, 0.3155],
        [0.2846, 0.3429, 0.3725],
        [0.1535, 0.4304, 0.4161],
        [0.2790, 0.2731, 0.4479],
        [0.2117, 0.3278, 0.4606],
        [0.2467, 0.4718, 0.2815],
        [0.2243, 0.3787, 0.3970],
        [0.2026, 0.3419, 0.4555],
        [0.2843, 0.2324, 0.4833],
        [0.2133, 0.3573, 0.4294],
        [0.2173, 0.3516, 0.4311],
        [0.2516, 0.3568, 0.3916],
        [0.3389, 0.3286, 0.3325],
        [0.2682, 0.3311, 0.4007],
        [0.1897, 0.3899, 0.4204],
        [0.2248, 0.2098, 0.5653],
        [0.283

## Train

In [ ]:
from tqdm import tqdm
EPOCHS = 2
optimizer = AdamW(model.parameters(), lr=8e-5)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0

  avg_loss = 0.0
  loop = tqdm(data_loader, leave=True)
  for d in loop:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    loop = tqdm(data_loader, leave=True)
    for d in loop:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
%%time

import time, sys


history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  )

  print('-' * 10)
  print(f'Train loss {train_loss} accuracy {train_acc}')
  print('-' * 10)

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_val)
  )
  print('-' * 10)
  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print('-' * 10)
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'sentiment_best_model_state.pt')
    best_accuracy = val_acc

Epoch 1/2
----------


  0%|          | 0/282 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length

----------
Train loss 1.0235577153821362 accuracy 0.527888888888889
----------


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length 

----------
Val   loss 1.0159979164600372 accuracy 0.532
----------

Epoch 2/2
----------


  0%|          | 0/282 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length

----------
Train loss 1.0071044490269736 accuracy 0.5385555555555556
----------


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length 

----------
Val   loss 0.9987255856394768 accuracy 0.542
----------

CPU times: user 11min 4s, sys: 8.21 s, total: 11min 12s
Wall time: 11min 17s


## Eval

In [ ]:
test_acc, _ = eval_model(
  model,
  test_data_loader,
  loss_fn,
  device,
  len(df_test)
)

test_acc.item()

  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length 

0.528

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()
  
  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      texts = d["comment_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs, real_values

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)
print(classification_report(y_test, y_pred, target_names=class_names))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

              precision    recall  f1-score   support

    negative       0.56      0.08      0.14       126
     neutral       0.00      0.00      0.00       115
    positive       0.53      0.98      0.69       259

    accuracy                           0.53       500
   macro avg       0.36      0.35      0.27       500
weighted avg       0.41      0.53      0.39       500



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
torch.save(model, 'sentimenttV1.pt')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/sentimentV1.pt /content/drive/MyDrive